In [1]:
#let's import

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans

!pip install folium
import folium

from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

print("imports is DONE")

imports is DONE


In [2]:
wiki_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
post_codes = requests.get(wiki_url).text

In [9]:
wiki_xml = BeautifulSoup(post_codes, 'xml')
wiki_table=wiki_xml.find('table')
column_names=['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns=column_names)

In [10]:
for tr_cell in wiki_table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
        df.head()

In [11]:
df

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [12]:
df=df[df['Borough']!='Not assigned']

In [13]:
df2=df[df['Neighborhood']=='Not assigned']
df2

,Postalcode,Borough,Neighborhood


##### There is no record with Neigborhood = "Not assigned", so nothing to do.

In [14]:
df

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [16]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_merged'},inplace=True)

    Postalcode                                Neighborhood_merged
0          M1B                                     Malvern, Rouge
1          M1C             Rouge Hill, Port Union, Highland Creek
2          M1E                  Guildwood, Morningside, West Hill
3          M1G                                             Woburn
4          M1H                                          Cedarbrae
..         ...                                                ...
98         M9N                                             Weston
99         M9P                                          Westmount
100        M9R  Kingsview Village, St. Phillips, Martin Grove ...
101        M9V  South Steeles, Silverstone, Humbergate, Jamest...
102        M9W                Northwest, West Humber - Clairville

[103 rows x 2 columns]


In [17]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [18]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [19]:
df_merge.drop_duplicates(inplace=True)

In [20]:
df_merge.rename(columns={'Neighborhood_merged':'Neighborhood'},inplace=True)

In [21]:
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [22]:
df_merge.shape

(103, 3)